In [ ]:
!pip install cdsapi

  Preparing metadata (setup.py) ... done
  Created wheel for multiurl: filename=multiurl-0.3.3-py3-none-any.whl size=21230 sha256=bacb1105379ef582df94a1a723835040a6d784767aeb2a156a21099031d81a8e
  Stored in directory: /root/.cache/pip/wheels/be/05/e0/65a6edb0a000498aeaefbadd80228bf5ed1bdbb82840ca1692
Successfully built multiurl


In [ ]:
import json

with open('processed-hurdat2.json', 'r') as f:
    hurdat2data = json.load(f)
# Now you can work with the data loaded from the JSON file.
# Example: print the keys of the first item in the data list
if isinstance(hurdat2data, list) and len(hurdat2data) > 0:
    print(hurdat2data)
else:
  print("The JSON data is empty or not a list.")


[{'storm_id': '1950', 'name': 'ABLE', 'num_records': 51, 'observations': [{'date': '1950-08-12', 'time': '00:00 UTC', 'latitude': 17.1, 'longitude': -55.5, 'wind_speed': 35, 'pressure': -999}, {'date': '1950-08-12', 'time': '06:00 UTC', 'latitude': 17.7, 'longitude': -56.3, 'wind_speed': 40, 'pressure': -999}, {'date': '1950-08-12', 'time': '12:00 UTC', 'latitude': 18.2, 'longitude': -57.4, 'wind_speed': 45, 'pressure': -999}, {'date': '1950-08-12', 'time': '18:00 UTC', 'latitude': 19.0, 'longitude': -58.6, 'wind_speed': 50, 'pressure': -999}, {'date': '1950-08-13', 'time': '00:00 UTC', 'latitude': 20.0, 'longitude': -60.0, 'wind_speed': 50, 'pressure': -999}, {'date': '1950-08-13', 'time': '06:00 UTC', 'latitude': 20.7, 'longitude': -61.1, 'wind_speed': 50, 'pressure': -999}, {'date': '1950-08-13', 'time': '12:00 UTC', 'latitude': 21.3, 'longitude': -62.2, 'wind_speed': 55, 'pressure': -999}, {'date': '1950-08-13', 'time': '18:00 UTC', 'latitude': 22.0, 'longitude': -63.2, 'wind_speed

In [ ]:
from datetime import datetime

focusYear = '2020'
yearsInHurricane = []

for i in hurdat2data:
    if i['storm_id'] == focusYear:
        yearsInHurricane.extend(i['observations'])

yearsInHurricane.sort(key=lambda x: datetime.strptime(f"{x['date']} {x['time']}", '%Y-%m-%d %H:%M UTC'))
yearsInHurricane

[{'date': '2020-05-16', 'time': '18:00 UTC', 'latitude': 28.0, 'longitude': -78.7, 'wind_speed': 30, 'pressure': 1008}, {'date': '2020-05-17', 'time': '00:00 UTC', 'latitude': 28.9, 'longitude': -78.0, 'wind_speed': 35, 'pressure': 1006}, {'date': '2020-05-17', 'time': '06:00 UTC', 'latitude': 29.6, 'longitude': -77.6, 'wind_speed': 35, 'pressure': 1004}, {'date': '2020-05-17', 'time': '12:00 UTC', 'latitude': 30.3, 'longitude': -77.5, 'wind_speed': 35, 'pressure': 1003}, {'date': '2020-05-17', 'time': '18:00 UTC', 'latitude': 31.0, 'longitude': -77.3, 'wind_speed': 40, 'pressure': 1003}, {'date': '2020-05-18', 'time': '00:00 UTC', 'latitude': 31.9, 'longitude': -77.0, 'wind_speed': 40, 'pressure': 1003}, {'date': '2020-05-18', 'time': '06:00 UTC', 'latitude': 33.1, 'longitude': -76.7, 'wind_speed': 40, 'pressure': 1002}, {'date': '2020-05-18', 'time': '12:00 UTC', 'latitude': 34.4, 'longitude': -75.9, 'wind_speed': 45, 'pressure': 1000}, {'date': '2020-05-18', 'time': '18:00 UTC', 'la

In [ ]:
from datetime import datetime

firstObservationDate = datetime.strptime(yearsInHurricane[0]['date'], '%Y-%m-%d')
lastObservationDate = datetime.strptime(yearsInHurricane[-1]['date'], '%Y-%m-%d')
print(firstObservationDate)
print(lastObservationDate)

2020-05-16 00:00:00
2020-11-18 00:00:00


In [ ]:
from datetime import datetime, timedelta

months = []
current_date = firstObservationDate
while current_date <= lastObservationDate:
    months.append(current_date.strftime("%m"))  # Extract the month as a string
    current_date += timedelta(days=31)  # Move to the next month (approximately)
    current_date = current_date.replace(day=1)  # Reset to the first day of the next month

# Remove duplicate months and ensure correct ordering
all_months = sorted(list(set(months)))

# Zero-pad the month strings to two places
zero_padded_months = [month.zfill(2) for month in all_months]

zero_padded_months

['05', '06', '07', '08', '09', '10', '11']

In [ ]:
from google.colab import userdata
apikey = userdata.get("CDS_TOKEN")
with open("/root/.cdsapirc", "w") as f:
    print("url: https://cds.climate.copernicus.eu/api", file=f)
    print(f"key: {apikey}", file=f)

In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["mean_sea_level_pressure"],
    "year": ["2020"],
    "month": zero_padded_months,
    "day": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12",
        "13", "14", "15",
        "16", "17", "18",
        "19", "20", "21",
        "22", "23", "24",
        "25", "26", "27",
        "28", "29", "30",
        "31"
    ],
    "time": [
        "00:00",
        "06:00",
        "12:00",
        "18:00"
    ],
    "data_format": "netcdf",
    "download_format": "zip",
    "area": [65, -120, 0, 0]
}

client = cdsapi.Client()
client.retrieve(dataset, request, "/content/data.zip")


2024-12-07 23:59:47,237 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
INFO:datapi.legacy_api_client:[2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-12-07 23:59:47,241 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.le

d061ded1fa39fc4e9aaa5c9647bed4ea.zip:   0%|          | 0.00/144M [00:00<?, ?B/s]

'/content/data.zip'

In [ ]:
# To redownload processed datasets

# import requests
# import os

# url = "https://object-store.os-api.cci2.ecmwf.int:443/cci2-prod-cache/cf48bdf4dd229f28543766368b03ca7b.zip"
# file_path = "/content/data.zip"


# response = requests.get(url, stream=True)
# response.raise_for_status()  # Raise an exception for bad status codes

# with open(file_path, "wb") as f:
#   for chunk in response.iter_content(chunk_size=8192): #8KB chunks
#     f.write(chunk)

# print(f"File downloaded to: {file_path}")

File downloaded to: /content/data.zip


In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: data_stream-oper_stepType-instant.nc  


In [ ]:
import xarray as xr

filename = 'data_stream-oper_stepType-instant.nc'

# Load the NetCDF dataset
ds = xr.open_dataset(filename)
ds

<xarray.Dataset> Size: 430MB
Dimensions:     (valid_time: 856, latitude: 261, longitude: 481)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 7kB 2020-05-01 ... 2020-11-30T18:...
  * latitude    (latitude) float64 2kB 65.0 64.75 64.5 64.25 ... 0.5 0.25 0.0
  * longitude   (longitude) float64 4kB -120.0 -119.8 -119.5 ... -0.5 -0.25 0.0
    expver      (valid_time) <U4 14kB ...
Data variables:
    msl         (valid_time, latitude, longitude) float32 430MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-08T00:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
if 'expver' in ds:
  ds = ds.drop_vars('expver')
  print("Column 'expver' dropped successfully.")
else:
  print("Column 'expver' not found in the dataset.")

if "number" in ds.keys():
  ds = ds.drop_vars('number')
  print("Column 'number' dropped successfully.")
else:
  print("Column 'number' not found in the dataset.")

# if "valid_time" in ds.keys():
#   ds["valid_time"] = ds["valid_time"].dt.strftime("%y/%m/%d %H:%M:%S")
#   print("Column 'valid_time' convert to string successfully.")
# else:
#   print("Column 'valid_time' not found in the dataset.")

ds

Column 'expver' dropped successfully.
Column 'number' dropped successfully.


<xarray.Dataset> Size: 430MB
Dimensions:     (valid_time: 856, latitude: 261, longitude: 481)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 7kB 2020-05-01 ... 2020-11-30T18:...
  * latitude    (latitude) float64 2kB 65.0 64.75 64.5 64.25 ... 0.5 0.25 0.0
  * longitude   (longitude) float64 4kB -120.0 -119.8 -119.5 ... -0.5 -0.25 0.0
Data variables:
    msl         (valid_time, latitude, longitude) float32 430MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-08T00:01 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
!mkdir /content/processed

In [ ]:
# Round latitude and longitude to the nearest degree
ds = ds.assign_coords(
    latitude=ds.latitude.round(0),
    longitude=ds.longitude.round(0)
)
# Group by the rounded coordinates and compute the mean temperature
ds = ds.groupby(['latitude', 'longitude']).mean()
print(ds)

<xarray.Dataset> Size: 27MB
Dimensions:     (latitude: 66, longitude: 121, valid_time: 856)
Coordinates:
  * latitude    (latitude) float64 528B 0.0 1.0 2.0 3.0 ... 62.0 63.0 64.0 65.0
  * longitude   (longitude) float64 968B -120.0 -119.0 -118.0 ... -2.0 -1.0 -0.0
  * valid_time  (valid_time) datetime64[ns] 7kB 2020-05-01 ... 2020-11-30T18:...
Data variables:
    msl         (valid_time, latitude, longitude) float32 27MB 1.009e+05 ... ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-08T00:01 GRIB to CDM+CF via cfgrib-0.9.1...


In [ ]:
df = ds.to_dataframe()
df = df.reset_index()
df

,latitude,longitude,valid_time,msl
0,0.0,-120.0,2020-05-01 00:00:00,100877.593750
1,0.0,-120.0,2020-05-01 06:00:00,101223.148438
2,0.0,-120.0,2020-05-01 12:00:00,100960.304688
3,0.0,-120.0,2020-05-01 18:00:00,101278.570312
4,0.0,-120.0,2020-05-02 00:00:00,100917.718750
...,...,...,...,...
6836011,65.0,-0.0,2020-11-29 18:00:00,100244.437500
6836012,65.0,-0.0,2020-11-30 00:00:00,99867.687500
6836013,65.0,-0.0,2020-11-30 06:00:00,99805.125000
6836014,65.0,-0.0,2020-11-30 12:00:00,100606.476562


In [ ]:
# Round the 'msl' column to the nearest whole number
df['msl'] = df['msl'].round(0)
df

,latitude,longitude,valid_time,msl
0,0.0,-120.0,2020-05-01 00:00:00,100878.0
1,0.0,-120.0,2020-05-01 06:00:00,101223.0
2,0.0,-120.0,2020-05-01 12:00:00,100960.0
3,0.0,-120.0,2020-05-01 18:00:00,101279.0
4,0.0,-120.0,2020-05-02 00:00:00,100918.0
...,...,...,...,...
6836011,65.0,-0.0,2020-11-29 18:00:00,100244.0
6836012,65.0,-0.0,2020-11-30 00:00:00,99868.0
6836013,65.0,-0.0,2020-11-30 06:00:00,99805.0
6836014,65.0,-0.0,2020-11-30 12:00:00,100606.0


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame with the 'valid_time' column
# and it's already in datetime format (if not, convert it first)

# Convert 'valid_time' to datetime objects if it's not already
# df['valid_time'] = df['valid_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Reference datetime (January 1, 2020, 00:00:00)
# reference_datetime = pd.to_datetime('2020-01-01 00:00:00')

# Calculate the difference in hours
# df['valid_time'] = (df['valid_time'] - reference_datetime).dt.total_seconds() / 3600

df

,latitude,longitude,valid_time,msl
0,0,-120,2020-05-01 00:00:00,100878
1,0,-120,2020-05-01 06:00:00,101223
2,0,-120,2020-05-01 12:00:00,100960
3,0,-120,2020-05-01 18:00:00,101279
4,0,-120,2020-05-02 00:00:00,100918
...,...,...,...,...
6836011,65,0,2020-11-29 18:00:00,100244
6836012,65,0,2020-11-30 00:00:00,99868
6836013,65,0,2020-11-30 06:00:00,99805
6836014,65,0,2020-11-30 12:00:00,100606


In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Convert all columns to int8, handling potential errors
for col in df.columns:
    try:
        df[col] = df[col].astype('int32')
    except (ValueError, TypeError):
        print(f"Could not convert column '{col}' to int8. Check data types.")
        # # Handle the error as needed, e.g., convert to a different type or skip
        # # Example: Convert to int32 if int8 fails
        # try:
        #     df[col] = df[col].astype('int32')
        # except (ValueError, TypeError):
        #   print(f"Could not convert column '{col}' to int32 either. Skipping conversion for this column.")
        #   pass # Or choose another alternative for handling errors.

df

Could not convert column 'valid_time' to int8. Check data types.


,latitude,longitude,valid_time,msl
0,0,-120,2020-05-01 00:00:00,100878
1,0,-120,2020-05-01 06:00:00,101223
2,0,-120,2020-05-01 12:00:00,100960
3,0,-120,2020-05-01 18:00:00,101279
4,0,-120,2020-05-02 00:00:00,100918
...,...,...,...,...
6836011,65,0,2020-11-29 18:00:00,100244
6836012,65,0,2020-11-30 00:00:00,99868
6836013,65,0,2020-11-30 06:00:00,99805
6836014,65,0,2020-11-30 12:00:00,100606


In [ ]:
df.to_csv("/content/processed/msl.csv", index=False)

In [ ]:
!pip install supabase

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0


In [ ]:
import os
from supabase import create_client, Client

table_name = "msl"

from google.colab import userdata
url = userdata.get("SUPABASE_URL")
key = userdata.get("SUPABASE_KEY")

supabase: Client = create_client(url, key)

In [ ]:
import pandas as pd
from supabase import create_client, Client

chunk_size = 100_000
num_chunks = (len(df) + chunk_size - 1) // chunk_size

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(df))
    chunk = df.iloc[start:end].to_dict(orient="records")
    try:
        response = supabase.table(table_name).insert(chunk).execute()
        print(f"Successfully inserted chunk {i+1} of {num_chunks}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Successfully inserted chunk 1 of 69
Successfully inserted chunk 2 of 69
Successfully inserted chunk 3 of 69
Successfully inserted chunk 4 of 69
Successfully inserted chunk 5 of 69
Successfully inserted chunk 6 of 69
Successfully inserted chunk 7 of 69
Successfully inserted chunk 8 of 69
Successfully inserted chunk 9 of 69
Successfully inserted chunk 10 of 69
Successfully inserted chunk 11 of 69
Successfully inserted chunk 12 of 69
Successfully inserted chunk 13 of 69
Successfully inserted chunk 14 of 69
Successfully inserted chunk 15 of 69
Successfully inserted chunk 16 of 69
Successfully inserted chunk 17 of 69
Successfully inserted chunk 18 of 69
Successfully inserted chunk 19 of 69
Successfully inserted chunk 20 of 69
Successfully inserted chunk 21 of 69
Successfully inserted chunk 22 of 69
Successfully inserted chunk 23 of 69
Successfully inserted chunk 24 of 69
Successfully inserted chunk 25 of 69
Successfully inserted chunk 26 of 69
Successfully inserted chunk 27 of 69
Successful

In [ ]:
!zip -r processed.zip /content/processed/

  adding: content/processed/chunk_0.csv (deflated 87%)


In [ ]:
!ls processed -lah

total 221M
drwxr-xr-x 2 root root 4.0K Dec  8 00:28 .
drwxr-xr-x 1 root root 4.0K Dec  8 00:07 ..
-rw-r--r-- 1 root root 221M Dec  8 00:28 msl.csv
